In [1]:
import asyncio 
import requests
import pandas as pd
import json
from urllib.parse import urlencode, urlunparse

In [2]:
def get_weather_data():
    df = pd.read_csv('coordinates.csv')
    df_weather = pd.DataFrame(columns=['city','code_insee','datetime','rr10','rr1','probarain','weather','weather_text','tmin','tmax','wind10m','gust10m'])

    for code_insee in df['code_insee'].unique():
        url = 'https://api.meteo-concept.com/api/forecast/daily?'
        params = {
            'token': "e3d5025f48dd7dd7109ef9323194feaad5f9f639e58cd8d77df7e700e49f0777",
            'insee': code_insee,
            
        }
        r = requests.get(url=url,params=params)

        list_forecast = r.json()['forecast']

        for forecast in list_forecast[1:8]:
            new_row = pd.DataFrame(
                {
                    'city' : [df[df['code_insee'] == code_insee]['city'].iloc[0]],
                    'code_insee' : [code_insee],
                    'datetime':[ forecast['datetime']] ,
                    'rr10' : [forecast['rr10']] ,
                    'rr1': [forecast['rr1']],
                    'probarain': [forecast['probarain']],
                    'weather': [forecast['weather']],
                    'weather_text': [''],
                    'tmin': [forecast['tmin']],
                    'tmax' : [forecast['tmax']],
                    'wind10m' : [forecast['wind10m']],
                    'gust10m' : [forecast['gust10m']]
                }
            )
            df_weather = pd.concat([df_weather,new_row],ignore_index=True)

    df_annex = pd.read_csv('weather_annex.csv',encoding='ISO-8859-1')
    mapping_dict = df_annex.set_index('Code temps')['Description'].to_dict()
    df_weather['weather_text'] = df_weather['weather'].map(mapping_dict)

    return df_weather


In [3]:

df_weather = get_weather_data()

In [4]:
df_weather = get_weather_data()

df_weather['rainy_day'] = 0
df_weather['cold_day'] = 0
df_weather['hot_day'] = 0
df_weather['windy_day'] = 0
df_weather['total_day'] = 0

df_weather.loc[df_weather['tmax'] >= 35, 'hot_day'] = 1
df_weather.loc[df_weather['tmax'] <20, 'cold_day'] = 1
df_weather.loc[df_weather['rr1'] >10, 'rainy_day'] = 1
df_weather.loc[df_weather['rr10'] >10, 'rainy_day'] = 1
df_weather.loc[df_weather['rr10'] >10, 'rainy_day'] = 1
df_weather.loc[df_weather['wind10m'] >50, 'windy_day'] = 1
df_weather.loc[df_weather['gust10m'] >50, 'windy_day'] = 1

df_weather['total_day'] =  df_weather[['hot_day', 'cold_day', 'rainy_day','windy_day']].max(axis=1)

df_weather




,city,code_insee,datetime,rr10,rr1,probarain,weather,weather_text,tmin,tmax,wind10m,gust10m,rainy_day,cold_day,hot_day,windy_day,total_day
0,Mont Saint Michel,50353,2024-08-12T02:00:00+0200,0,0,10,3,Nuageux,19,26,15,45,0,0,0,0,0
1,Mont Saint Michel,50353,2024-08-13T02:00:00+0200,0,0,10,3,Nuageux,16,25,15,30,0,0,0,0,0
2,Mont Saint Michel,50353,2024-08-14T02:00:00+0200,0.2,0.2,30,40,Averses de pluie locales et faibles,16,22,15,35,0,0,0,0,0
3,Mont Saint Michel,50353,2024-08-15T02:00:00+0200,0.2,0.2,50,40,Averses de pluie locales et faibles,14,22,15,23,0,0,0,0,0
4,Mont Saint Michel,50353,2024-08-16T02:00:00+0200,3.1,8.9,60,40,Averses de pluie locales et faibles,15,21,20,29,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,La Rochelle,17300,2024-08-14T02:00:00+0200,1.2,3.2,70,3,Nuageux,19,21,15,36,0,0,0,0,0
241,La Rochelle,17300,2024-08-15T02:00:00+0200,0,0,40,3,Nuageux,17,23,15,28,0,0,0,0,0
242,La Rochelle,17300,2024-08-16T02:00:00+0200,0.2,0.2,50,40,Averses de pluie locales et faibles,17,23,20,29,0,0,0,0,0
243,La Rochelle,17300,2024-08-17T02:00:00+0200,0.4,0.4,50,40,Averses de pluie locales et faibles,17,22,20,30,0,0,0,0,0


In [158]:
df_weather[df_weather['city']=='Biarritz']

,city,code_insee,datetime,rr10,rr1,probarain,weather,weather_text,tmin,tmax,wind10m,gust10m,rainy_day,cold_day,hot_day,total_day,windy_day
224,Biarritz,64122,2024-08-11T02:00:00+0200,0,0,10,1,Peu nuageux,23,41,15,56,0,0,1,1,1
225,Biarritz,64122,2024-08-12T02:00:00+0200,2.2,3.5,70,4,Très nuageux,22,23,15,31,0,0,0,0,0
226,Biarritz,64122,2024-08-13T02:00:00+0200,4.7,7,80,40,Averses de pluie locales et faibles,20,22,10,28,0,0,0,0,0
227,Biarritz,64122,2024-08-14T02:00:00+0200,4.1,18,60,41,Averses de pluie locales,17,22,10,20,1,0,0,1,0
228,Biarritz,64122,2024-08-15T02:00:00+0200,0.6,0.6,60,40,Averses de pluie locales et faibles,17,22,10,20,0,0,0,0,0
229,Biarritz,64122,2024-08-16T02:00:00+0200,0,0,40,3,Nuageux,16,23,10,18,0,0,0,0,0
230,Biarritz,64122,2024-08-17T02:00:00+0200,0.4,4,60,41,Averses de pluie locales,17,22,10,18,0,0,0,0,0


In [7]:
df_aggregated = df_weather.groupby('city')[['total_day','rainy_day','cold_day','hot_day','windy_day','rr10', 'rr1', 'tmin', 'tmax','wind10m','gust10m']].agg(['sum', 'mean']).reset_index()
df_aggregated.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df_aggregated.columns.values]
df_aggregated=  df_aggregated.sort_values(by=['total_day_sum','rr10_mean','tmax_mean'],ignore_index=True)

columns_to_remove = ['total_day_mean','rainy_day_mean','cold_day_mean','hot_day_mean','windy_day_mean','tmin_sum','tmax_sum','wind10m_sum','gust10m_sum','rr10_sum','rr1_sum']
for column in columns_to_remove:
    del df_aggregated[column] 

df_aggregated['id'] = df_aggregated.index + 1

df_aggregated


,city,total_day_sum,rainy_day_sum,cold_day_sum,hot_day_sum,windy_day_sum,rr10_mean,rr1_mean,tmin_mean,tmax_mean,wind10m_mean,gust10m_mean,id
0,Marseille,0,0,0,0,0,0.0,0.0,23.0,28.142857,14.285714,28.285714,1
1,Cassis,0,0,0,0,0,0.0,0.0,23.0,28.428571,14.285714,28.428571,2
2,Bormes les Mimosas,0,0,0,0,0,0.0,0.0,23.857143,29.0,13.571429,30.142857,3
3,Aix en Provence,0,0,0,0,0,0.0,0.0,22.571429,30.857143,12.857143,27.857143,4
4,Collioure,0,0,0,0,0,0.142857,0.428571,21.857143,27.428571,17.857143,30.714286,5
5,Ariege,0,0,0,0,0,0.342857,1.142857,15.142857,26.0,13.571429,29.571429,6
6,Gorges du Verdon,0,0,0,0,0,0.342857,1.142857,15.142857,26.0,13.571429,29.571429,7
7,Saintes Maries de la mer,0,0,0,0,0,0.414286,0.785714,22.428571,28.571429,16.428571,32.285714,8
8,La Rochelle,0,0,0,0,0,0.557143,0.842857,18.142857,22.857143,17.857143,32.714286,9
9,Mont Saint Michel,0,0,0,0,0,0.585714,1.671429,15.285714,22.571429,15.714286,30.571429,10


In [15]:
def get_city_ranking(df_weather,tmin=20,tmax=35,rainmax=10,windmax=50):

    #Add a coloum to know if a day is rainy/cold/hot/windy. The ranking will be done based on the number of days that have a low number of days not in the scipe
    df_weather['rainy_day'] = 0
    df_weather['cold_day'] = 0
    df_weather['hot_day'] = 0
    df_weather['windy_day'] = 0
    df_weather['total_day'] = 0

    df_weather.loc[df_weather['tmax'] >= tmax, 'hot_day'] = 1
    df_weather.loc[df_weather['tmax'] <tmin, 'cold_day'] = 1
    df_weather.loc[df_weather['rr1'] >rainmax, 'rainy_day'] = 1
    df_weather.loc[df_weather['rr10'] >rainmax, 'rainy_day'] = 1
    df_weather.loc[df_weather['wind10m'] >windmax, 'windy_day'] = 1
    df_weather.loc[df_weather['gust10m'] >windmax, 'windy_day'] = 1

    df_weather['total_day'] =  df_weather[['hot_day', 'cold_day', 'rainy_day','windy_day']].max(axis=1)

    df_aggregated = df_weather.groupby('city')[['total_day','rainy_day','cold_day','hot_day','windy_day','rr10', 'rr1', 'tmin', 'tmax','wind10m','gust10m']].agg(['sum', 'mean']).reset_index()
    df_aggregated.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df_aggregated.columns.values]
    df_aggregated=  df_aggregated.sort_values(by=['total_day_sum','rr10_mean','tmax_mean'],ignore_index=True)

    columns_to_remove = ['total_day_mean','rainy_day_mean','cold_day_mean','hot_day_mean','windy_day_mean','tmin_sum','tmax_sum','wind10m_sum','gust10m_sum','rr10_sum','rr1_sum']
    for column in columns_to_remove:
        del df_aggregated[column] 

    df_aggregated['id'] = df_aggregated.index + 1

    return df_aggregated


In [19]:
df_ranking = get_city_ranking(df_weather,20,25)
df_ranking

,city,total_day_sum,rainy_day_sum,cold_day_sum,hot_day_sum,windy_day_sum,rr10_mean,rr1_mean,tmin_mean,tmax_mean,wind10m_mean,gust10m_mean,id
0,La Rochelle,1,0,0,1,0,0.557143,0.842857,18.142857,22.857143,17.857143,32.714286,1
1,St Malo,1,0,0,1,1,0.571429,1.657143,15.428571,21.285714,16.428571,32.571429,2
2,Biarritz,1,0,0,0,1,1.0,1.657143,18.714286,23.142857,12.142857,27.714286,3
3,Bayonne,1,0,0,0,1,1.042857,1.728571,18.571429,23.428571,11.428571,25.0,4
4,Mont Saint Michel,2,0,0,2,0,0.585714,1.671429,15.285714,22.571429,15.714286,30.571429,5
5,Amiens,2,0,0,2,0,1.357143,2.342857,16.285714,24.285714,13.571429,29.285714,6
6,Lille,2,0,0,2,0,1.528571,2.985714,17.0,24.0,14.285714,30.714286,7
7,Ariege,3,0,0,3,0,0.342857,1.142857,15.142857,26.0,13.571429,29.571429,8
8,Gorges du Verdon,3,0,0,3,0,0.342857,1.142857,15.142857,26.0,13.571429,29.571429,9
9,Paris,3,0,0,3,0,1.257143,1.757143,17.714286,26.142857,10.714286,23.428571,10
